<a href="https://colab.research.google.com/github/AUGUSTO664/SE_svd/blob/master/Grades_Prediction_using_PT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
import pandas as pd
import seaborn as sns
import torch 
import torch.nn as nn
import math

In [3]:
#We have the grades of the data students 
#we want to predict the a grade based on the grades before
#we will work with english and softskilss
df = pd.read_excel("calificaciones 1.xlsx")
df

,nombre,clave,quimica_basica,fisica,fundamentos_algoritmos,arquitectura_computadoras,programacion,estructura_datos,paradigmas_programacion,introduccion_ingenieria_datos,preprocesamiento_datos,ingles_uno,ingles_dos,Ingles_3,algebra_lineal,funciones_matematicas,probabilidad_estadistica,matematicas_discretas,calculo_diferencial,modelos_estadisticos,expresion_oral_escrita,desarrollo_humano,inteligencia_emocional
0,Moyer,1,9,7,9,9,8,9,10,7,9,8,8,10,7,8,9,9,9,9,8,7,8
1,Karla,2,10,9,9,10,10,9,9,8,10,8,8,8,8,10,9,9,10,9,8,10,9
2,Jason,3,10,9,10,10,10,10,10,9,10,9,9,10,10,10,10,10,10,10,9,10,10
3,Pedro,4,9,9,9,10,10,10,10,9,10,9,9,10,9,10,10,10,10,10,9,10,10
4,Hurtado,5,10,9,9,10,10,10,10,9,10,9,9,10,9,10,10,9,10,9,10,10,10
5,Gabo,6,9,9,9,10,9,8,9,8,9,9,9,10,8,9,9,9,10,9,9,9,9
6,Efrain,7,10,10,8,10,10,9,9,8,9,8,8,9,8,10,9,9,10,9,9,10,9
7,Andress,8,10,9,7,9,10,8,9,8,7,8,9,9,8,9,8,7,9,9,9,9,8
8,Jorge,9,9,9,8,10,9,8,9,8,8,9,9,8,9,10,8,8,10,9,8,9,9
9,Osiris,10,10,9,10,10,10,10,10,9,9,9,9,10,9,10,10,10,10,10,10,10,10


In [14]:
#now we to extract the grades for English and Soft-skills

#we are going to crete the independent and the objective for english
independent = df[["ingles_uno", "ingles_dos"]].values
objective = df["Ingles_3"].values

#creating the tensors
X = torch.tensor(independent, dtype=torch.float)
y = torch.tensor(objective, dtype=torch.float)
#the variable xPredicted is a single input for which we want to predict
#a grade using the parameters learned by the neural network 
xPredicted = torch.tensor(([7,9]), dtype=torch.float)

In [11]:
#Checking the tensor's size 
print(X.size())
print(y.size())

torch.Size([14, 2])
torch.Size([14])


In [15]:
#Scale units 
X_max, _ = torch.max(X,0)
xPredicted_max, _ = torch.max(xPredicted,0)

X = torch.div(X, X_max)
xPredicted = torch.div(xPredicted, xPredicted_max)
y = y / 100 #max test score is 100
print(xPredicted)

tensor([0.7778, 1.0000])


In [29]:
#Model (computation Graph)
class Neural_Network(nn.Module):
  def __init__(self,):
    super(Neural_Network,self).__init__()
    #Parameters 
    #TODO: Parameters can be parameterized intead of declaring the here
    self.inputSize = 2
    self.outputSize = 1
    self.hiddenSize = 3

    #weights
    self.w1 = torch.randn(self.inputSize, self.hiddenSize)
    self.w2 = torch.randn(self.hiddenSize, self.outputSize)

  def forward(self, X):
    self.z = torch.matmul(X,self.w1)
    self.z2 = self.sigmoid(self.z)#activation function
    self.z3 = torch.matmul(self.z2, self.w2)
    o = self.sigmoid(self.z3) #Final activation function 

  def sigmoid(self, s):
    return 1/(1 + torch.exp(-s))

  def sigmoidPrime(self, s):
    #derivative of sigmoid
    return s*(1-s)


  def backward(self, X, y, o):
    self.o_error = y-o #error input
    self.o_delta = self.o_error * self.sigmoidPrime(o)#derivative of sig to error
    self.z2_error = torch.matmul(self.o_delta, torch.t(self.w2))
    self.z2_delta = self.z2_error * self.sigmoidPrime(self.z2)
    self.w1 += torch.matmul(torch.t(X), self.z2_delta)
    self.w2 += torch.matmul(torch.t(self.z2), self.o_delta)

  def train(self, X, y):
    #forward + backward pass for training
    o = self.forward(X)
    self.backward(X,y,o)

  def saveWeights(self, model):
    #we will use the pytorch internal storage functions
    torch.save(model, "NN")
    #you can reload the model with all the weigths and so forth with: 
    #torch.load("NN")
    

  def predict(self):
    print("Predicted data based on trained weights: ")
    print("Input (scaled): \n"+str(xPredicted))
    print("Output: \n "+str(self.forward(xPredicted)))




In [30]:
#Training 
NN = Neural_Network()

In [35]:
for i in range(1000):
  if(i%100) == 0:
    #mean sum squared loss
    #print ("#" + str(i) + " Loss: " + str(torch.mean((y - NN(X))**2).detach().item()))
    NN.train(X,y)
NN.saveWeights(NN)
NN.predict()

print("Finished training!")

TypeError: ignored

In [40]:
class Neural_Network2(nn.Module):
    def __init__(self, ):
        super(Neural_Network2, self).__init__()
        # parameters
        # TODO: parameters can be parameterized instead of declaring them here
        self.inputSize = 2
        self.outputSize = 1
        self.hiddenSize = 3
        
        # weights
        self.W1 = torch.randn(self.inputSize, self.hiddenSize) # 3 X 2 tensor
        self.W2 = torch.randn(self.hiddenSize, self.outputSize) # 3 X 1 tensor
        
    def forward(self, X):
        self.z = torch.matmul(X, self.W1) # 3 X 3 ".dot" does not broadcast in PyTorch
        self.z2 = self.sigmoid(self.z) # activation function
        self.z3 = torch.matmul(self.z2, self.W2)
        o = self.sigmoid(self.z3) # final activation function
        return o
        
    def sigmoid(self, s):
        return 1 / (1 + torch.exp(-s))
    
    def sigmoidPrime(self, s):
        # derivative of sigmoid
        return s * (1 - s)
    
    def backward(self, X, y, o):
        self.o_error = y - o # error in output
        self.o_delta = self.o_error * self.sigmoidPrime(o) # derivative of sig to error
        self.z2_error = torch.matmul(self.o_delta, torch.t(self.W2))
        self.z2_delta = self.z2_error * self.sigmoidPrime(self.z2)
        self.W1 += torch.matmul(torch.t(X), self.z2_delta)
        self.W2 += torch.matmul(torch.t(self.z2), self.o_delta)
        
    def train(self, X, y):
        # forward + backward pass for training
        o = self.forward(X)
        self.backward(X, y, o)
        
    def saveWeights(self, model):
        # we will use the PyTorch internal storage functions
        torch.save(model, "NN2")
        # you can reload model with all the weights and so forth with:
        # torch.load("NN")
        
    def predict(self):
        print ("Predicted data based on trained weights: ")
        print ("Input (scaled): \n" + str(xPredicted))
        print ("Output: \n" + str(self.forward(xPredicted)))

In [45]:
NN2 = Neural_Network2()
for i in range(1000):  # trains the NN 1,000 times
    #if (i % 100) == 0:
        #print ("#" + str(i) + " Loss: " + str(torch.mean((y - NN(X))**2).detach().item()))  # mean sum squared loss
    NN2.train(X, y)
NN2.saveWeights(NN2)
NN2.predict()

print("Finished training!")

RuntimeError: ignored